In [57]:
from bs4 import BeautifulSoup as bs
import re 
import os
import lxml
import xml.etree.ElementTree as ET
from lxml import objectify
from datetime import datetime
import numpy as np 
import pandas as pd 
import requests
import ftplib
from io import BytesIO
import tqdm

### Prepare to load data

In [58]:
ftp_url = "oda.ft.dk"
ftp_dir = "/ODAXML/Referat/samling/"


##### Tomme lister

In [59]:
dirs = []
sub_dir = []
filer = []
filer_navne = []
sti_navn = []
indhold = []

suppe = []

### Login til ftp serveren og henter en liste over biblioteker

In [60]:
ftp = ftplib.FTP(ftp_url)
ftp.login("anonymous", "wpg345@alumni.ku.dk")
ftp.set_pasv(True)
ftp.cwd(ftp_dir)
ftp.dir(dirs.append)

### Bygger en liste med underbibliotekter

In [61]:
for i in range (0, len(dirs), 1):
    sub_dir.append(ftp_dir+dirs[i][-5:]+'/') 

### Bygger en liste med filnavne

In [62]:
for i in range (0, len(sub_dir), 1):
    ftp.cwd(sub_dir[i])
    ftp.dir(filer.append)

### Ekstrakt af filnavn

In [63]:
for i in range(0, len(filer)):
    filer_navne.append(filer[i][39:])

### Opbygning af hele stien og filnavn

In [64]:
for i in range(0, len(filer_navne)):
    sti_navn.append(ftp_dir+filer_navne[i][0:5]+'/'+filer_navne[i])

### Overførsel af de enkelte filer fra ftp://oda.ft.dk, og indholdet skrives til liste

In [65]:
for i in tqdm.tqdm(range(0, len(sti_navn))):
# for i in tqdm.tqdm(range(0, 10)):
    r = BytesIO()
    ftp.retrbinary(f"RETR {sti_navn[i]}", r.write)
    indhold.append(r.getvalue())
    r.close()

100%|██████████| 1489/1489 [02:53<00:00,  8.58it/s]


# Define metadata function

In [69]:
def extract_metadata(xml): 
    soup = bs(xml, 'lxml')
    
    d = soup.find("dateofsitting").text[:10]
    m = soup.find("meetingnumber").text
    s = soup.find("parliamentarysession").text
    g = soup.find("parliamentarygroup").text
    l = soup.find("location").text
    
    date.append(d)
    meetingnumber.append(m)
    session.append(s)
    group.append(g)
    location.append(l)

In [92]:
# empty lists (columns)
date = []
content = []
meetingnumber = []
session = []
group = []
location = []

# Apply metadata function to files

In [93]:
xml_sample = indhold[0:5]

for xml in xml_sample: 
    extract_metadata(xml)

In [105]:
meetingnumber

['100', '101', '102', '103', '106']

# Content function 

In [107]:
def extract_content(xml):
    soup = bs(xml, 'lxml')
    
    dp = soup.find_all("dagsordenpunkt")
    
    for punkt in range(0, len(dp)):
        content.append(dp[punkt].get_text(separator = ' '))

# Apply content function to files

In [108]:
for xml in xml_sample: 
    extract_content(xml)

# 

# Make DataFrame

In [46]:
df = pd.DataFrame([date, content, meetingnumber, session, group, location]).transpose()
df.columns = ['date', 'content', 'meetingnumber', 'session', 'group', 'location']
df

,date,content,meetingnumber,session,group,location
0,2017-10-27,0 FM Punkt 0 Meddelelser fra formanden Holger ...,11,20171,Folketinget,Folketingssalen
1,2022-02-10,1 222 L BEH1 1. behandling af L 222: Om nedsæt...,60,20211,Folketinget,Folketingssalen
2,2018-01-18,2 218 L BEH1 1. behandling af L 218: Om nedsæt...,46,20171,Folketinget,Folketingssalen
3,None,3 219 L BEH1 1. behandling af L 219: Om forhøj...,None,None,None,None
4,None,4 223 L BEH1 1. behandling af L 223: Om staten...,None,None,None,None
5,None,5 221 L BEH1 1. behandling af L 221: Om person...,None,None,None,None
6,None,6 220 L BEH1 1. behandling af L 220: Om nedsæt...,None,None,None,None
7,None,0 FM Punkt 0 Meddelelser fra formanden Holger ...,None,None,None,None
